In [29]:
import numpy as np
import pandas as pd
from sklearn import tree
import os
import io
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

training = "C:\\Extras\\DataScience\\Hackathon1\\train.csv"
train_df = pd.read_csv(training, header = 0)

final = "C:\\Extras\\DataScience\\Hackathon1\\test.csv"
final_df = pd.read_csv(final, header = 0)

# Creating new Variables

In [30]:
#To check the first and last alphabet of the word is a valid entry

train_df['LW'] = train_df['Word'].str[-1:]
train_df['FW'] = train_df['Word'].str[:1]

train_df['LW_chk'] = np.where(((train_df['LW'] == ("(",")","'",",","`","?",";","-",":","/","’","-","[","]")) |
                              train_df['LW'].str[0].str.isalnum()), 1, 0)
train_df['FW_chk'] = np.where(((train_df['FW'] == ("(",")","'",",","`","?",";","-",":","/","’","-","[","]")) |
                              train_df['FW'].str[0].str.isalnum()), 1, 0)

In [31]:
length=train_df.groupby('Sent_ID', as_index=False).agg("count")
train_df = train_df.merge(length, on = "Sent_ID", how="left").drop(['Word_y','Doc_ID_y','tag_y','LW_y','FW_y',
                                                                         'LW_chk_y','FW_chk_y'],axis = 1)

In [32]:
train_df = train_df.rename(columns = {"id_x": "id", "Doc_ID_x":"Doc_ID", "Word_x": "Word",
                                               "tag_x": "tag", "LW_x":"LW", "FW_x": "FW",
                                               "LW_chk_x": "LW_chk", "FW_chk_x": "FW_chk", "id_y":"count_sent"})

In [33]:
train_df['counter']=1
train_df['counter'] = train_df.groupby('Sent_ID').counter.cumsum()

In [34]:
# To get the word's location in the sentence

train_df['word_location']=train_df['counter']/train_df['count_sent']

In [35]:
train_df.head()

,id,Doc_ID,Sent_ID,Word,tag,LW,FW,LW_chk,FW_chk,count_sent,counter,word_location
0,1,1,1,Obesity,O,y,O,1,1,15,1,0.066667
1,2,1,1,in,O,n,i,1,1,15,2,0.133333
2,3,1,1,Low-,O,-,L,0,1,15,3,0.200000
3,4,1,1,and,O,d,a,1,1,15,4,0.266667
4,5,1,1,Middle-Income,O,e,M,1,1,15,5,0.333333


# TRAINING: Multinomial NB

In [36]:
train_df["word_mapping"]=train_df["word_location"].apply(lambda x: "SW" if x<=0.01 else("LW" if x>=0.99 else "MW"))

In [37]:
train_df['words_combined'] = train_df[['Word', 'word_mapping']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

In [38]:
train_df['Word']=train_df['words_combined']

In [39]:
from sklearn.cross_validation import train_test_split
train_df=train_df.fillna(0)
train, test= train_test_split(train_df, test_size=0.3)

In [40]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(train['Word'].values.astype('U'))

classifier = MultinomialNB()
targets = train['tag'].values
targets_test=test['tag'].values

classifier.fit(counts, targets)

test_counts = vectorizer.transform(test['Word'].values.astype('U'))
predictions = classifier.predict(test_counts)

In [41]:
scoring_results= classifier.score(test_counts, targets_test)
scoring_results

0.98300040347724027

In [42]:
test_df=test.copy(deep=True)
test_df["tag"]=predictions

In [43]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from copy import deepcopy

from collections import namedtuple

# Evaluation metric for Innoplexus NER Challenge

def collect_named_entities(tokens): # Helper Function for score calculation
    """
    Creates a list of Entity named-tuples, storing the entity type and the start and end
    offsets of the entity.
    :param tokens: a list of labels
    :return: a list of Entity named-tuples
    """
    Entity = namedtuple("Entity", "e_type start_offset end_offset")
    named_entities = []
    start_offset = None
    end_offset = None
    ent_type = None

    for offset, token_tag in enumerate(tokens):

        if token_tag == 'O':
            if ent_type is not None and start_offset is not None:
                end_offset = offset - 1
                named_entities.append(Entity(ent_type, start_offset, end_offset))
                start_offset = None
                end_offset = None
                ent_type = None

        elif ent_type is None:
            ent_type = token_tag[2:]
            start_offset = offset

        elif ent_type != token_tag[2:] or (ent_type == token_tag[2:] and token_tag[:1] == 'B'):

            end_offset = offset - 1
            named_entities.append(Entity(ent_type, start_offset, end_offset))

            # start of a new entity
            ent_type = token_tag[2:]
            start_offset = offset
            end_offset = None

    # catches an entity that goes up until the last token
    if ent_type and start_offset and end_offset is None:
        named_entities.append(Entity(ent_type, start_offset, len(tokens)-1))

    return named_entities

def compute_metrics(true_named_entities, pred_named_entities): # Helper Function for score calculation
    eval_metrics = {'correct': 0, 'partial': 0, 'missed': 0, 'spurius': 0}
    target_tags_no_schema = ['indications']

    # overall results
    evaluation = {'partial': deepcopy(eval_metrics)}


    true_which_overlapped_with_pred = []  # keep track of entities that overlapped

    # go through each predicted named-entity
    for pred in pred_named_entities:
        found_overlap = False

        # check if there's an exact match, i.e.: boundary and entity type match
        if pred in true_named_entities:
            true_which_overlapped_with_pred.append(pred)
            evaluation['partial']['correct'] += 1

        else:

            # check for overlaps with any of the true entities
            for true in true_named_entities:

                
                # 2. check for an overlap i.e. not exact boundary match, with true entities
                if pred.start_offset <= true.end_offset and true.start_offset <= pred.end_offset:

                    true_which_overlapped_with_pred.append(true)

                    evaluation['partial']['partial'] += 1

                    found_overlap = True
                    break

            # count spurius (i.e., False Positive) entities
            if not found_overlap:
                # overall results
                evaluation['partial']['spurius'] += 1

    # count missed entities (i.e. False Negative)
    for true in true_named_entities:
        if true in true_which_overlapped_with_pred:
            continue
        else:
            # overall results
            evaluation['partial']['missed'] += 1

    # Compute 'possible', 'actual'
    for eval_type in ['partial']:

        correct = evaluation[eval_type]['correct']
        partial = evaluation[eval_type]['partial']
        missed = evaluation[eval_type]['missed']
        spurius = evaluation[eval_type]['spurius']

        # possible: nr. annotations in the gold-standard which contribute to the final score
        evaluation[eval_type]['possible'] = correct + partial + missed

        # actual: number of annotations produced by the NER system
        evaluation[eval_type]['actual'] = correct + partial + spurius

        actual = evaluation[eval_type]['actual']
        possible = evaluation[eval_type]['possible']

    return evaluation

def list_converter(df): # Helper Function for score calculation
    keys, values = df.sort_values('Sent_ID_x').values.T
    ukeys, index = np.unique(keys,True)
    lists = [list(array) for array in np.split(values,index[1:])]
    return lists

# ideal and pred respectively represent dataframes containing actual labels and predictions for the set of sentences in the test data. 
# It has the same format as the sample submission (id, Sent_ID, tag)

def calculate_score(ideal, pred): # Calculates the final F1 Score

    merged = ideal.merge(pred, on = "id", how="inner").drop(['Sent_ID_y'],axis = 1)
    
    
    # The scores are calculated sentence wise and then aggregated to calculate the overall score, for this
    # List converter function groups the labels by sentence to generate a list of lists with each inner list representing a sentence in sequence
    ideal_ = list_converter(merged.drop(['id','tag_y'],axis = 1))
    pred_ = list_converter(merged.drop(['id','tag_x'],axis = 1))

    metrics_results = {'correct': 0, 'partial': 0,
                   'missed': 0, 'spurius': 0, 'possible': 0, 'actual': 0}

    results = {'partial': deepcopy(metrics_results)}


    for true_ents, pred_ents in zip(ideal_, pred_):    
    # compute results for one sentence
        tmp_results = compute_metrics(collect_named_entities(true_ents),collect_named_entities(pred_ents))
    
    # aggregate overall results
        for eval_schema in results.keys():
            for metric in metrics_results.keys():
                results[eval_schema][metric] += tmp_results[eval_schema][metric]
    correct = results['partial']['correct']
    partial = results['partial']['partial']
    missed = results['partial']['missed']
    spurius = results['partial']['spurius']
    actual = results['partial']['actual']
    possible = results['partial']['possible']


    precision = (correct + 0.5 * partial) / actual if actual > 0 else 0
    recall = (correct + 0.5 * partial) / possible if possible > 0 else 0


    score = (2 * precision * recall)/(precision + recall) if (precision + recall) >0 else 0
    
    # final score
    return score

In [44]:
test_df2=test_df[['id','Sent_ID','tag']]
test2=test[['id','Sent_ID','tag']]
calculate_score(test2,test_df2)

0.4304018153641798

In [45]:
merged = test.merge(test_df, on = "id", how="inner").drop(['Sent_ID_y','Word_y','Doc_ID_y'],axis = 1)
check=merged.groupby(['tag_x', 'tag_y']).agg(['count'])
check['id','count']

tag_x          tag_y        
B-indications  B-indications       5043
               I-indications        552
               O                  10411
I-indications  B-indications        774
               I-indications       1833
               O                  10798
O              B-indications        407
               I-indications        231
               O                1333101
Name: (id, count), dtype: int64

In [46]:
test_df['tag'] = np.where(test_df['LW_chk']==0, 'O', test_df['tag'])
test_df['tag'] = np.where(test_df['FW_chk']==0, 'O', test_df['tag'])

In [47]:
test_df2=test_df[['id','Sent_ID','tag']]
test2=test[['id','Sent_ID','tag']]
calculate_score(test2,test_df2)

0.4303590709005841

In [48]:
merged = test.merge(test_df, on = "id", how="inner").drop(['Sent_ID_y','Word_y','Doc_ID_y'],axis = 1)
check=merged.groupby(['tag_x', 'tag_y']).agg(['count'])
check['id','count']

tag_x          tag_y        
B-indications  B-indications       5039
               I-indications        552
               O                  10415
I-indications  B-indications        774
               I-indications       1833
               O                  10798
O              B-indications        404
               I-indications        223
               O                1333112
Name: (id, count), dtype: int64

In [49]:
merged.head()

,id,Doc_ID_x,Sent_ID_x,Word_x,tag_x,LW_x,FW_x,LW_chk_x,FW_chk_x,count_sent_x,...,tag_y,LW_y,FW_y,LW_chk_y,FW_chk_y,count_sent_y,counter_y,word_location_y,word_mapping_y,words_combined_y
0,1993916,13270,84248,defectMW,I-indications,t,d,1,1,9,...,O,t,d,1,1,9,8,0.888889,MW,defectMW
1,492633,3288,20785,treesMW,O,s,t,1,1,27,...,O,s,t,1,1,27,18,0.666667,MW,treesMW
2,3699630,24463,155711,weaknessesMW,O,s,w,1,1,51,...,O,s,w,1,1,51,38,0.745098,MW,weaknessesMW
3,835859,5559,35167,locationMW,O,n,l,1,1,16,...,O,n,l,1,1,16,8,0.500000,MW,locationMW
4,4197872,27740,176808,sindMW,O,d,s,1,1,14,...,O,d,s,1,1,14,5,0.357143,MW,sindMW


# TESTING THE FINAL DATA

In [50]:
final_df['LW'] = final_df['Word'].str[-1:]
final_df['FW'] = final_df['Word'].str[:1]
final_df['LW_chk'] = np.where(((final_df['LW'] == ("(",")","'",",","`","?",";","-",":","/","’","-","[","]")) |
                              final_df['LW'].str[0].str.isalnum()), 1, 0)
final_df['FW_chk'] = np.where(((final_df['FW'] == ("(",")","'",",","`","?",";","-",":","/","’","-","[","]")) |
                              final_df['FW'].str[0].str.isalnum()), 1, 0)

length=final_df.groupby('Sent_ID', as_index=False).agg("count")
final_df = final_df.merge(length, on = "Sent_ID", how="left").drop(['Word_y','Doc_ID_y','LW_y','FW_y',
                                                                         'LW_chk_y','FW_chk_y'],axis = 1)

final_df = final_df.rename(columns = {"id_x": "id", "Doc_ID_x":"Doc_ID", "Word_x": "Word",
                                                "LW_x":"LW", "FW_x": "FW",
                                               "LW_chk_x": "LW_chk", "FW_chk_x": "FW_chk", "id_y":"count_sent"})

final_df['counter']=1
final_df['counter'] = final_df.groupby('Sent_ID').counter.cumsum()

final_df['word_location']=final_df['counter']/final_df['count_sent']

In [51]:
final_df.head()

,id,Doc_ID,Sent_ID,Word,LW,FW,LW_chk,FW_chk,count_sent,counter,word_location
0,4543834,30001,191283,CCCVA,A,C,1,1,8,1,0.125
1,4543835,30001,191283,",",",",",",0,0,8,2,0.250
2,4543836,30001,191283,MANOVA,A,M,1,1,8,3,0.375
3,4543837,30001,191283,",",",",",",0,0,8,4,0.500
4,4543838,30001,191283,my,y,m,1,1,8,5,0.625


In [52]:
final_df["word_mapping"] = final_df["word_location"].apply(lambda x: "SW" if x<=0.01 else("LW" if x>=0.99 else "MW"))
final_df['words_combined'] = final_df[['Word', 'word_mapping']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

In [53]:
final_df['Word']=final_df['words_combined']

In [54]:
final_counts = vectorizer.transform(final_df['Word'].values.astype('U'))
predictions_2 = classifier.predict(final_counts)

final_df["tag"]=predictions_2
final_df2=final_df[['id','Sent_ID','tag']]

In [57]:
final_df['tag'] = np.where(final_df['LW_chk']==0, 'O', final_df['tag'])
final_df['tag'] = np.where(final_df['FW_chk']==0, 'O', final_df['tag'])
final_df2=final_df[['id','Sent_ID','tag']]

final_df2.to_csv('C:\\Extras\\DataScience\\Hackathon1\\final_sub\\sample_submission.csv', index = False)